In [90]:
import numpy as np
import pandas as pd
import os
import plotnine as p9
import warnings
warnings.filterwarnings('ignore', module='plotnine')
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
meta = "../data/raw/meta_open.csv"
test_dir = "tests"
results_files = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if f.endswith("results.csv")]
df_bdgp = pd.read_csv(meta, usecols=['uid', 'primaryspaceusage'])
df_bdgp = df_bdgp.rename(columns={"uid": "buildingid"})

# Preview of meta_open.csv
meat_open.csv uses `primaryspaceusage` for defining property type

| uid                | dataend        | datastart   | energystarscore | heatingtype | industry            | mainheatingtype | numberoffloors | occupants | primaryspaceusage           | rating | sqft   | sqm         | subindustry              | timezone            | yearbuilt | nickname  | primaryspaceuse_abbrev | newweatherfilename |
|--------------------|----------------|-------------|-----------------|-------------|---------------------|-----------------|----------------|-----------|-----------------------------|--------|--------|-------------|--------------------------|---------------------|-----------|-----------|------------------------|--------------------|
| PrimClass_Everett  | 31/12/12 23:00 | 1/1/12 0:00 |                 |             | Education           |                 |                |           | Primary/Secondary Classroom |        | 105530 | 9804.05359  | Primary/Secondary School | America/New_York    |           | Everett   | PrimClass              | weather12.csv      |
| UnivClass_Clifford | 31/12/15 23:00 | 1/1/15 0:00 |                 |             | Education           |                 |                |           | College Classroom           |        | 56969  | 5292.591007 | College/University       | America/New_York    | 1967      | Clifford  | UnivClass              | weather2.csv       |
| Office_Elizabeth   | 31/12/12 23:00 | 1/1/12 0:00 |                 |             | Commercial Property |                 |                |           | Office                      |        | 294651 | 27373.96185 | Commercial Real Estate   | America/Los_Angeles |           | Elizabeth | Office                 | weather22.csv      |
| Office_Ellie       | 31/12/12 23:00 | 1/1/12 0:00 |                 |             | Commercial Property |                 |                |           | Office                      |        | 496517 | 46127.91885 | Bank/Financial Services  | America/Los_Angeles |           | Ellie     | Office                 | weather28.csv      |

---

# Preview of a Results File

results files use `buildingtype`

| buildingid      | yearbuilt | buildingtype     | numberofstories | squarefootage | state      | zipcode | latitude   | longitude   | consumption_actual | consumption_model |
|-----------------|-----------|------------------|-----------------|---------------|------------|---------|------------|-------------|--------------------|-------------------|
| UnivLab_Alexis  | 1967      | Office           | 1               | 49384         | Arizona    | 85287   | 33.4172    | -111.9365   | 2957596.977        | 1502520.33        |
| UnivLab_Cecil   | 1974      | Office           | 1               | 125507        | New York   | 14853   | 42.447577  | -76.4760889 | 5040775.791        | 11655361.01       |
| UnivLab_Clodagh | 1933      | Office           | 1               | 28432         | New York   | 14853   | 42.4494115 | -76.4786409 | 1235479.616        | 1363849.014       |
| UnivDorm_Poppy  |           | MidriseApartment | 1               | 113000        | New Jersey | 8540    | 40.35      | -74.7       | 18459999.01        | 2733363.631       |

In [151]:
# read in all results files and store as a list
def gen_results(results_files):
    df2 = pd.DataFrame()
    for f in results_files:
        df = pd.read_csv(f)
        df2 = df2.append(df, ignore_index=True)
    return df2

def my_plot2(df, property_type, facets=False):
    lims = [0, max(max(df["consumption_actual"]), max(df["consumption_model"]))]
    p = p9.ggplot(data=df, mapping=p9.aes(x="consumption_actual", y="consumption_model")) #, color='buildingtype'))
    p = p + p9.geom_point(p9.aes(color='buildingtype', size='numberofstories'), alpha=0.5)
    if facets:
        p = p + p9.facets.facet_wrap('~state', labeller='label_both')
    p = p + p9.geom_line(p9.aes(y='consumption_actual'), color='black') + \
            p9.geom_smooth(p9.aes(color='buildingtype'), method='lm', se=False) +\
            p9.labs(title=property_type, x="Actual Consumption, Million kbtu", y="Modeled Consumption, Million kbtu")
    n = "tests/figs/" + property_type.replace(' ', '_').replace('/', '_') + '.png'
    p9.ggplot.save(p, filename=n, width=12, height=9, units='in', dpi=500)

In [152]:
def make_plots(df_bdgp, results_files):
    bdgp_unique_types = df_bdgp.primaryspaceusage.unique()
    df2 = pd.DataFrame()
    df_results = gen_results(results_files)
    df_results['consumption_actual'] = df_results['consumption_actual'].apply(lambda x: x*(10**-6))
    df_results['consumption_model'] = df_results['consumption_model'].apply(lambda x: x*(10**-6))
    for property_type in bdgp_unique_types:
        df = pd.merge(df_results, df_bdgp, on="buildingid")
        df = df.loc[df['primaryspaceusage'] == property_type]
        plt.clf()
        my_plot2(df, property_type)

In [153]:
make_plots(df_bdgp, results_files)

/Users/cmosiman/.pyenv/versions/3.6.5/envs/jupyter-seat/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/cmosiman/.pyenv/versions/3.6.5/envs/jupyter-seat/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/cmosiman/.pyenv/versions/3.6.5/envs/jupyter-seat/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/cmosiman/.pyenv/versions/3.6.5/envs/jupyter-seat/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.

<Figure size 864x864 with 0 Axes>

In [94]:
regressor = LinearRegression()
x_t = df["consumption_actual"].values.reshape(-1,1)
y_t = df["consumption_model"].values.reshape(-1,1)
regressor.fit(X=x_t, y=y_t)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [100]:
regressor.score(x_t, y_t)
bdgp_unique_types = df_bdgp.primaryspaceusage.unique()
df_results = gen_results(results_files)
df = pd.merge(df_results, df_bdgp, on="buildingid")

In [101]:
df.head(20)

,buildingid,yearbuilt,buildingtype,numberofstories,squarefootage,state,zipcode,latitude,longitude,consumption_actual,consumption_model,cvrmseelec,nmbeelec,primaryspaceusage
0,PrimClass_Josue,1966.0,SecondarySchool,1,17050.0176,Wales,NaN,51.526783,-3.328933,2.229060e+05,1.129864e+06,449.674008,-443.869924,Primary/Secondary Classroom
1,PrimClass_Josue,1966.0,PrimarySchool,1,17050.0176,Wales,NaN,51.526783,-3.328933,2.229060e+05,1.100226e+06,419.057841,-429.361536,Primary/Secondary Classroom
2,PrimClass_Josue,1966.0,Office,1,17050.0176,Wales,NaN,51.526783,-3.328933,2.229060e+05,8.525994e+05,303.933250,-308.175016,Primary/Secondary Classroom
3,PrimClass_Ellen,NaN,SecondarySchool,1,79503.0000,Tennessee,37186.0,36.575496,-86.227413,1.778696e+05,4.373863e+06,2508.961392,-2573.484184,Primary/Secondary Classroom
4,PrimClass_Ellen,NaN,PrimarySchool,1,79503.0000,Tennessee,37186.0,36.575496,-86.227413,1.778696e+05,4.726859e+06,2693.474385,-2789.982339,Primary/Secondary Classroom
5,PrimClass_Ellen,NaN,Office,1,79503.0000,Tennessee,37186.0,36.575496,-86.227413,1.778696e+05,3.511264e+06,1976.076273,-2044.439311,Primary/Secondary Classroom
6,UnivClass_Serenity,1994.0,SecondarySchool,1,31118.4349,England,NaN,50.937801,-1.394014,8.738562e+05,1.863589e+06,130.076917,-123.556636,College Classroom
7,UnivClass_Serenity,1994.0,PrimarySchool,1,31118.4349,England,NaN,50.937801,-1.394014,8.738562e+05,1.874432e+06,123.547539,-124.910847,College Classroom
8,UnivClass_Serenity,1994.0,Office,1,31118.4349,England,NaN,50.937801,-1.394014,8.738562e+05,1.402144e+06,67.997776,-65.950559,College Classroom
9,PrimClass_Joel,1976.0,SecondarySchool,1,13939.2505,Wales,NaN,51.473588,-3.172932,1.937056e+05,9.745930e+05,445.347449,-439.776966,Primary/Secondary Classroom


# Garbage Code

In [66]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
%matplotlib inline
# rcParams['figure.figsize'] = 12,12
# sns.set()
sns.set(rc={'figure.figsize':(12,12)})
plt.style.use("ggplot")

def my_plot(df, property_type):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot()
#     g = sns.relplot(x="consumption_actual", y="consumption_model", hue="buildingtype", 
#                     style="buildingtype", data=df)
#     g.plot(X=df["consumption_actual"], Y=df["consumption_actual"])
#     plt.title(property_type)
#     g.fig.add_title(property_type)
#     ax.add_title(property_type)
#     p.show()
#     plt.title(property_type)
#     plt.show()

def ax_selector(i):
    if i in [0,1,2]:
        return([0,i])
    elif i in [3,4]:
        return([1,i-3])
    
#         df.plot.scatter(x='consumption_actual', y='consumption_model', c='buildingtype')
#         df.plot.


def my_plot(df, property_type):
        lims = [0, max(max(df["consumption_actual"]), max(df["consumption_model"]))]
        sns.lmplot(x="consumption_actual", y="consumption_model", scatter=False, hue="buildingtype", data=df)
        sns.scatterplot(x="consumption_actual", y="consumption_model", hue="buildingtype", data=df, alpha=0.5, legend=False)
        sns_fig = sns.lineplot(x="consumption_actual", y="consumption_actual", data=df, color='black', legend=False)
        plt.title(property_type)
        plt.xlabel("Actual Consumption, Million kbtu")
#         plt.xlim(lims)
#         plt.ylim(lims)
        plt.ylabel("Modeled Consumption, Million kbtu")
        n = "tests/figs/" + property_type.replace(' ', '_').replace('/', '_') + '.png'
        plt.show()
        sns_fig.figure.savefig(n)
#         plt.savefig(n)
#         plt.show()